In [8]:
import numpy as np
import pandas as pd
import pickle

In [28]:
data = pd.read_csv('./../data/2003.csv')

In [5]:
rdata = pd.read_csv('./../data/round.csv')

In [6]:
rdata.head(2)

,Tournament_Year,Tournament_#,Permanent_Tournament_#,Course_#,Player_Number,Player_Name,Round_Number,Tee_Time,Round_Score,End_of_Event_Pos._(text),Finishing_Pos,Finishing_Pct
0,2003,10,16,656,1320,"Estes, Bob",1,11:50,66,T6,7.5,0.197917
1,2003,10,16,656,1361,"Forsman, Dan",1,11:00,69,T21,22.0,0.600694


In [7]:
data.head(2)

,Year,Tourn.#,Player_#,Course_#,Permanent_Tournament_#,Player_First_Name,Player_Last_Name,Round,Tournament_Name,Course_Name,...,Went_to_X,Went_to_Y,Went_to_Z,Started_at_X,Started_at_Y,Started_at_Z,Distance_from_hole,Cat,Green_to_work_with,Difficulty_Start
0,2003,440,20593,538,47,Greg,Chalmers,5,Las Vegas Invitational,TPC Summerlin,...,-398.204081,-104.551714,4.269,-1269.931111,-429.453390,NaN,1340.580188,Tee Box,37.363389,4.215190
1,2003,70,6515,500,7,Shaun,Micheel,1,Nissan Open,Riviera CC,...,-316.155150,-378.347489,-14.323,-639.244982,-1131.828038,NaN,1299.872631,Tee Box,21.707906,4.457746


In [10]:
with open('./../PickleFiles/hole_tups.pkl','r') as pickleFile:
    tournorder = pickle.load(pickleFile)

In [30]:
df = pd.DataFrame(tournorder)
df.columns = ['Year','Permanent_Tournament_#','Round','Course_#','Hole']
round_tups = df[['Year','Course_#','Round']].drop_duplicates().values.tolist()

In [31]:
round_tups

[[2003, 656, 1],
 [2003, 656, 2],
 [2003, 656, 3],
 [2003, 656, 4],
 [2003, 6, 2],
 [2003, 6, 1],
 [2003, 6, 3],
 [2003, 6, 4],
 [2003, 510, 2],
 [2003, 510, 4],
 [2003, 510, 1],
 [2003, 510, 3],
 [2003, 649, 2],
 [2003, 649, 1],
 [2003, 649, 5],
 [2003, 649, 4],
 [2003, 649, 3],
 [2003, 5, 3],
 [2003, 5, 4],
 [2003, 4, 1],
 [2003, 4, 2],
 [2003, 4, 4],
 [2003, 4, 3],
 [2003, 500, 1],
 [2003, 500, 2],
 [2003, 500, 3],
 [2003, 500, 4],
 [2003, 101, 1],
 [2003, 101, 2],
 [2003, 101, 4],
 [2003, 101, 3],
 [2003, 8, 1],
 [2003, 8, 2],
 [2003, 8, 4],
 [2003, 8, 3],
 [2003, 692, 4],
 [2003, 692, 1],
 [2003, 692, 2],
 [2003, 692, 3],
 [2003, 9, 2],
 [2003, 9, 1],
 [2003, 9, 3],
 [2003, 9, 4],
 [2003, 11, 2],
 [2003, 11, 1],
 [2003, 11, 4],
 [2003, 11, 3],
 [2003, 638, 2],
 [2003, 638, 1],
 [2003, 638, 4],
 [2003, 638, 3],
 [2003, 12, 2],
 [2003, 12, 3],
 [2003, 12, 1],
 [2003, 12, 4],
 [2003, 699, 3],
 [2003, 699, 2],
 [2003, 699, 4],
 [2003, 699, 1],
 [2003, 511, 4],
 [2003, 511, 2],
 [2003,